# 1 Carga de datos

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_excel("/home/clisueno/output/results2025-22042025.xlsx", sheet_name="basal", dtype=str)

In [4]:
df.dtypes

nombre_paciente              object
edad_paciente                object
medida_edad_paciente         object
id_paciente                  object
eps_paciente                 object
fecha_proced                 object
escala_epworth               object
tiempo_dormido               object
tiempo_en_cama               object
eficiencia_sueno             object
porc_sueno_rem               object
porcentaje_sueno_profundo    object
ausencia_sueno_profundo      object
indice_microalertamientos    object
iah                          object
gravedad_iah                 object
ido                          object
tiempo_bajo_90so2            object
ct90                         object
iac                          object
iao                          object
iam                          object
ih                           object
fuente                       object
dtype: object

# 2 Asignación de tipos

## 2.1 Declaración de tipos

In [5]:
variables_str = [
    'nombre_paciente',
    'medida_edad_paciente',
    'id_paciente',
    'eps_paciente',
    'ausencia_sueno_profundo'
]

variables_int = [
    'edad_paciente',
    'escala_epworth',
]

variables_float = [
    'tiempo_dormido',
    'tiempo_en_cama',
    'eficiencia_sueno',
    'porc_sueno_rem',
    'porcentaje_sueno_profundo',
    'indice_microalertamientos',
    'iah',
    'ido',
    'tiempo_bajo_90so2',
    'ct90',
    'iac',
    'iao',
    'iam',
    'ih',
]

variables_datetime = ['fecha_proced']

## 2.2 Definición de funciones

In [6]:
import pandas as pd
from datetime import datetime
import numpy as np
import warnings

# Variables DATETIME ***********************************

def validar_transformar_datetime(df, variables):
    """
    Valida y transforma columnas de un DataFrame a datetime, reportando los resultados.

    Args:
        df (pd.DataFrame): El DataFrame a procesar.
        variables (list): Lista de nombres de columnas a transformar.

    Returns:
        dict: Diccionario con los resultados de la validación y transformación.
    """
    # Inicializar listas de resultados
    variables_no_encontradas = []
    variables_transformadas = []
    variables_no_transformadas = []
    variables_con_warnings = []

    # Lista de formatos permitidos
    formatos = ["%Y-%m-%d %H:%M:%S", "%m/%d/%Y"]

    def convertir_fecha(fecha_str):
        """Intenta convertir un valor a datetime utilizando varios formatos o retorna NaT si no es posible."""
        if pd.isna(fecha_str) or str(fecha_str).strip() in ["", "nan", "None"]:
            return pd.NaT
        for formato in formatos:
            try:
                return datetime.strptime(str(fecha_str).strip(), formato)
            except ValueError:
                continue
        return pd.NaT

    for variable in variables:
        if variable not in df.columns:
            variables_no_encontradas.append(variable)
        else:
            try:
                with warnings.catch_warnings(record=True) as w:
                    warnings.simplefilter("always", UserWarning)

                    # Aplicar la conversión
                    df[variable] = df[variable].apply(convertir_fecha)

                    # Verificar advertencias
                    if len(w) > 0:
                        variables_con_warnings.append(variable)

                    # Verificar si todos los valores son NaT tras la transformación
                    if df[variable].isna().all():
                        variables_no_transformadas.append(variable)
                    else:
                        variables_transformadas.append(variable)

            except Exception:
                variables_no_transformadas.append(variable)

    # Generar el reporte
    print("\n************ Resultado de la transformación **************\n")
    print(f"{len(variables_no_encontradas)} Variables no encontradas: {variables_no_encontradas}")
    print(f"{len(variables_no_transformadas)} Variables no transformadas: {variables_no_transformadas}")
    print(f"{len(variables_con_warnings)} Variables transformadas con warnings: {variables_con_warnings}")
    print(f"{len(variables_transformadas)} Variables transformadas exitosamente: {variables_transformadas}")

    return {
        "no_encontradas": variables_no_encontradas,
        "no_transformadas": variables_no_transformadas,
        "con_warnings": variables_con_warnings,
        "transformadas": variables_transformadas
    }

# Validación y transformación a INTEGER *****************************

def validar_transformar_int(df, variables):
    """
    Valida y transforma columnas en un DataFrame a enteros, usando pd.NA para valores no válidos.

    Args:
        df (pd.DataFrame): DataFrame a validar y transformar.
        variables (list): Lista de nombres de las columnas a validar y transformar.

    Returns:
        dict: Diccionario con los resultados de la validación y transformación.
    """
    variables_no_encontradas = []
    variables_transformadas = []
    variables_no_transformadas = []
    variables_con_warnings = []

    for var in variables:
        if var not in df.columns:
            variables_no_encontradas.append(var)
            print(f"La columna '{var}' no se encuentra en el DataFrame.")
        else:
            try:
                with warnings.catch_warnings(record=True) as w:
                    warnings.simplefilter("always", UserWarning)

                    # Convertir a numérico con soporte para nulos
                    df[var] = pd.to_numeric(df[var], errors='coerce')

                    # Cambiar dtype a entero con soporte de nulos
                    df[var] = df[var].astype('Int64')

                    if len(w) > 0:
                        variables_con_warnings.append(var)

                    if df[var].isna().all():
                        variables_no_transformadas.append(var)
                    else:
                        variables_transformadas.append(var)
            except Exception as e:
                variables_no_transformadas.append(var)
                print(f"Error al transformar la columna '{var}' a enteros: {e}")

    print("\n************ Resultado de la transformación **************\n")
    print(f"{len(variables_no_encontradas)} Variables no encontradas: {variables_no_encontradas}")
    print(f"{len(variables_no_transformadas)} Variables no transformadas: {variables_no_transformadas}")
    print(f"{len(variables_con_warnings)} Variables transformadas con warnings: {variables_con_warnings}")
    print(f"{len(variables_transformadas)} Variables transformadas exitosamente: {variables_transformadas}")

    return {
        "no_encontradas": variables_no_encontradas,
        "no_transformadas": variables_no_transformadas,
        "con_warnings": variables_con_warnings,
        "transformadas": variables_transformadas
    }

# Validación y transformación a FLOAT ************************************************

def validar_transformar_float(df, variables):
    """
    Valida y transforma columnas en un DataFrame a tipo float, usando NaN para valores no válidos.

    Args:
        df (pd.DataFrame): DataFrame a validar y transformar.
        variables (list): Lista de nombres de las columnas a validar y transformar.

    Returns:
        dict: Diccionario con las listas de variables no encontradas, transformadas exitosamente, no transformadas y transformadas con warnings.
    """
    variables_no_encontradas = []
    variables_transformadas = []
    variables_no_transformadas = []
    variables_con_warnings = []

    for var in variables:
        if var not in df.columns:
            variables_no_encontradas.append(var)
            print(f"La columna '{var}' no se encuentra en el DataFrame.")
        else:
            try:
                with warnings.catch_warnings(record=True) as w:
                    warnings.simplefilter("always", UserWarning)

                    # Convertir a float, usando NaN para valores inválidos
                    df[var] = pd.to_numeric(df[var], errors='coerce').astype(float)

                    if len(w) > 0:
                        variables_con_warnings.append(var)

                    if df[var].isna().all():
                        variables_no_transformadas.append(var)
                    else:
                        variables_transformadas.append(var)

            except Exception as e:
                variables_no_transformadas.append(var)
                print(f"Error al transformar la columna '{var}' a flotante: {e}")

    print("\n************ Resultado de la transformación **************\n")
    print(f"{len(variables_no_encontradas)} Variables no encontradas: {variables_no_encontradas}")
    print(f"{len(variables_no_transformadas)} Variables no transformadas: {variables_no_transformadas}")
    print(f"{len(variables_con_warnings)} Variables transformadas con warnings: {variables_con_warnings}")
    print(f"{len(variables_transformadas)} Variables transformadas exitosamente: {variables_transformadas}")

    return {
        "no_encontradas": variables_no_encontradas,
        "no_transformadas": variables_no_transformadas,
        "con_warnings": variables_con_warnings,
        "transformadas": variables_transformadas
    }

# Validación y transformación a STRING ****************************************

def validar_transformar_str(df, variables):
    """
    Valida y transforma columnas en un DataFrame a tipo str, conservando los NaN como valores faltantes.

    Args:
        df (pd.DataFrame): DataFrame a validar y transformar.
        variables (list): Lista de nombres de las columnas a validar y transformar.

    Returns:
        dict: Diccionario con los resultados de la transformación.
    """
    import warnings
    import pandas as pd

    variables_no_encontradas = []
    variables_transformadas = []
    variables_no_transformadas = []
    variables_con_warnings = []

    for var in variables:
        if var not in df.columns:
            variables_no_encontradas.append(var)
            print(f"La columna '{var}' no se encuentra en el DataFrame.")
        else:
            try:
                with warnings.catch_warnings(record=True) as w:
                    warnings.simplefilter("always", UserWarning)

                    df[var] = df[var].astype("string")  # usa tipo string (acepta pd.NA)

                    if len(w) > 0:
                        variables_con_warnings.append(var)

                    if df[var].isna().all():
                        variables_no_transformadas.append(var)
                    else:
                        variables_transformadas.append(var)

            except Exception as e:
                variables_no_transformadas.append(var)
                print(f"Error al transformar la columna '{var}' a cadena: {e}")

    print("\n************ Resultado de la transformación **************\n")
    print(f"{len(variables_no_encontradas)} Variables no encontradas: {variables_no_encontradas}")
    print(f"{len(variables_no_transformadas)} Variables no transformadas: {variables_no_transformadas}")
    print(f"{len(variables_con_warnings)} Variables transformadas con warnings: {variables_con_warnings}")
    print(f"{len(variables_transformadas)} Variables transformadas exitosamente: {variables_transformadas}")

    return {
        "no_encontradas": variables_no_encontradas,
        "no_transformadas": variables_no_transformadas,
        "con_warnings": variables_con_warnings,
        "transformadas": variables_transformadas
    }


## 2.3 Aplicación

In [7]:
validar_transformar_int(df,variables_int)


************ Resultado de la transformación **************

0 Variables no encontradas: []
0 Variables no transformadas: []
0 Variables transformadas con warnings: []
2 Variables transformadas exitosamente: ['edad_paciente', 'escala_epworth']


{'no_encontradas': [],
 'no_transformadas': [],
 'con_warnings': [],
 'transformadas': ['edad_paciente', 'escala_epworth']}

In [8]:
validar_transformar_float(df,variables_float)


************ Resultado de la transformación **************

0 Variables no encontradas: []
0 Variables no transformadas: []
0 Variables transformadas con warnings: []
14 Variables transformadas exitosamente: ['tiempo_dormido', 'tiempo_en_cama', 'eficiencia_sueno', 'porc_sueno_rem', 'porcentaje_sueno_profundo', 'indice_microalertamientos', 'iah', 'ido', 'tiempo_bajo_90so2', 'ct90', 'iac', 'iao', 'iam', 'ih']


{'no_encontradas': [],
 'no_transformadas': [],
 'con_warnings': [],
 'transformadas': ['tiempo_dormido',
  'tiempo_en_cama',
  'eficiencia_sueno',
  'porc_sueno_rem',
  'porcentaje_sueno_profundo',
  'indice_microalertamientos',
  'iah',
  'ido',
  'tiempo_bajo_90so2',
  'ct90',
  'iac',
  'iao',
  'iam',
  'ih']}

In [9]:
validar_transformar_datetime(df,variables_datetime)


************ Resultado de la transformación **************

0 Variables no encontradas: []
0 Variables no transformadas: []
0 Variables transformadas con warnings: []
1 Variables transformadas exitosamente: ['fecha_proced']


{'no_encontradas': [],
 'no_transformadas': [],
 'con_warnings': [],
 'transformadas': ['fecha_proced']}

In [10]:
validar_transformar_str(df,variables_str)


************ Resultado de la transformación **************

0 Variables no encontradas: []
0 Variables no transformadas: []
0 Variables transformadas con warnings: []
5 Variables transformadas exitosamente: ['nombre_paciente', 'medida_edad_paciente', 'id_paciente', 'eps_paciente', 'ausencia_sueno_profundo']


{'no_encontradas': [],
 'no_transformadas': [],
 'con_warnings': [],
 'transformadas': ['nombre_paciente',
  'medida_edad_paciente',
  'id_paciente',
  'eps_paciente',
  'ausencia_sueno_profundo']}

In [11]:
df.dtypes

nombre_paciente              string[python]
edad_paciente                         Int64
medida_edad_paciente         string[python]
id_paciente                  string[python]
eps_paciente                 string[python]
fecha_proced                 datetime64[ns]
escala_epworth                        Int64
tiempo_dormido                      float64
tiempo_en_cama                      float64
eficiencia_sueno                    float64
porc_sueno_rem                      float64
porcentaje_sueno_profundo           float64
ausencia_sueno_profundo      string[python]
indice_microalertamientos           float64
iah                                 float64
gravedad_iah                         object
ido                                 float64
tiempo_bajo_90so2                   float64
ct90                                float64
iac                                 float64
iao                                 float64
iam                                 float64
ih                              

# 3 EDA

## 3.1 Sumamrize dataframe

### a. Definición

In [15]:
from IPython.display import display, HTML

def format_large_numbers(value):
    """
    Formatea valores numéricos.
    - Valores mayores o menores a ±1E+06 se muestran en notación científica.
    - Otros valores se redondean a dos decimales.
    """
    if isinstance(value, (int, float)):
        if abs(value) > 1e6:
            return f"{value:.2E}"
        return f"{value:.2f}"
    return value  # Retorna el valor original si no es numérico


def summarize_statistics(df):
    """
    Extrae estadísticas básicas (mean, min, max) y las formatea,
    aplicando el cálculo solo a columnas numéricas.
    """
    # Seleccionamos solo las columnas numéricas
    df_numeric = df.select_dtypes(include=[np.number])
    
    # Reemplazamos los valores -9999 por NaN en las columnas numéricas
    df_numeric = df_numeric.replace({-9999: np.nan})
    
    # Calculamos las estadísticas básicas sobre las columnas numéricas
    stats = df_numeric.describe().loc[['mean', 'min', 'max']]
    
    # Usamos apply con map para evitar FutureWarning de applymap
    formatted_stats = stats.apply(lambda col: col.map(format_large_numbers))
    
    return formatted_stats


def calculate_missing_values(df):
    """
    Calcula la cantidad y porcentaje de valores faltantes.
    """
    missing_values = df.isnull().sum()
    missing_percentage = (missing_values / len(df)) * 100
    return pd.DataFrame({
        'Missing Values': missing_values,
        'Missing %': missing_percentage.map(lambda x: f"{x:.2f}%")
    })


def calculate_unique_values(df):
    """
    Calcula la cantidad y porcentaje de valores únicos.
    """
    unique_values = df.nunique()
    unique_percentage = (unique_values / len(df)) * 100
    return pd.DataFrame({
        'Unique Values': unique_values,
        'Unique %': unique_percentage.map(lambda x: f"{x:.2f}%")
    })


def extract_sample_values(df):
    """
    Extrae un valor de muestra por columna, excluyendo NaN y ceros.
    """
    samples = []
    for col in df.columns:
        non_na_values = df[col].dropna()
        non_zero_values = non_na_values[non_na_values != 0]
        if not non_zero_values.empty:
            samples.append(non_zero_values.iloc[0])  # Primer valor válido
        else:
            samples.append("N/A")
    return samples


def calculate_category_frequencies(df):
    """
    Calcula la frecuencia de valores en columnas categóricas,
    ignorando variables que contienen 'fecha'.
    Limita a 10 categorías y agrupa las restantes en 'OTROS'.
    """
    frequencies = []
    for col in df.columns:
        if df[col].dtype == 'object' or df[col].dtype.name == 'category':
            if 'fecha' not in col.lower():
                # Calcular frecuencias
                freq = df[col].value_counts()

                # Separar las 10 más frecuentes y agrupar el resto en "OTROS"
                top_10 = freq[:10]
                others_sum = freq[10:].sum()
                if others_sum > 0:
                    top_10 = pd.concat([top_10, pd.Series({'OTROS': others_sum})])

                # Formatear como cadena concatenada
                freq_str = '"{}"'.format(' | '.join([f"{k}: {v}" for k, v in top_10.items()]))

                frequencies.append(freq_str)
            else:
                frequencies.append("N/A")
        else:
            frequencies.append("N/A")
    return frequencies


def summarize_dataframe(df):
    """
    Genera un resumen detallado de un DataFrame.
    """
    variables = df.columns.tolist()
    summary_stats = summarize_statistics(df)
    data_types = df.dtypes
    missing_info = calculate_missing_values(df)
    unique_info = calculate_unique_values(df)
    sample_values = extract_sample_values(df)
    category_frequencies = calculate_category_frequencies(df)

    # Crear DataFrame resumen
    summary_df = pd.DataFrame({
        'Variable': variables,
        'Data Type': [data_types[col] for col in variables],
        'Count': [df[col].count() for col in variables],
        'Unique Values': [f"{unique_info.loc[col, 'Unique Values']} ({unique_info.loc[col, 'Unique %']})" for col in variables],
        'Category Frequencies': category_frequencies,
        'Missing Values': [f"{missing_info.loc[col, 'Missing Values']} ({missing_info.loc[col, 'Missing %']})" for col in variables],
        'Min': [summary_stats[col]['min'] if col in summary_stats.columns else "N/A" for col in variables],
        'Mean': [summary_stats[col]['mean'] if col in summary_stats.columns else "N/A" for col in variables],
        'Max': [summary_stats[col]['max'] if col in summary_stats.columns else "N/A" for col in variables],
        'Sample': sample_values
    })

    # Mostrar como HTML
    display(HTML(summary_df.to_html(index=False)))


### b. Aplicación

In [16]:
summarize_dataframe(df)

Variable,Data Type,Count,Unique Values,Category Frequencies,Missing Values,Min,Mean,Max,Sample
nombre_paciente,string[python],2711,2697 (99.48%),N/A,0 (0.00%),N/A,N/A,N/A,CARLOS ALBERTO DUARTE BERNAL
edad_paciente,Int64,2711,97 (3.58%),N/A,0 (0.00%),1.00,53.16,173.00,35
medida_edad_paciente,string[python],2709,2 (0.07%),N/A,2 (0.07%),N/A,N/A,N/A,anos
id_paciente,string[python],2711,2690 (99.23%),N/A,0 (0.00%),N/A,N/A,N/A,1095802388
eps_paciente,string[python],2710,107 (3.95%),N/A,1 (0.04%),N/A,N/A,N/A,Sanitas EPS
fecha_proced,datetime64[ns],2711,108 (3.98%),N/A,0 (0.00%),N/A,N/A,N/A,2025-04-02 00:00:00
escala_epworth,Int64,2471,25 (0.92%),N/A,240 (8.85%),0.00,7.48,24.00,1
tiempo_dormido,float64,2711,652 (24.05%),N/A,0 (0.00%),0.00,371.15,565.50,343.5
tiempo_en_cama,float64,2710,1404 (51.79%),N/A,1 (0.04%),48.10,463.59,693.00,469.0
eficiencia_sueno,float64,2709,527 (19.44%),N/A,2 (0.07%),4.20,80.17,99.50,73.2


# 4. Carga a BQ

## 4.1 Esquema BQ

### Instalar bibliotectas Google

### Definir esquema

In [12]:
from google.cloud import bigquery

schemaj = [
    bigquery.SchemaField('nombre_paciente', 'STRING', mode='NULLABLE'),
    bigquery.SchemaField('edad_paciente', 'INTEGER', mode='NULLABLE'),
    bigquery.SchemaField('medida_edad_paciente', 'STRING', mode='NULLABLE'),
    bigquery.SchemaField('id_paciente', 'STRING', mode='NULLABLE'),
    bigquery.SchemaField('eps_paciente', 'STRING', mode='NULLABLE'),
    bigquery.SchemaField('fecha_proced', 'DATETIME', mode='NULLABLE'),
    bigquery.SchemaField('escala_epworth', 'INTEGER', mode='NULLABLE'),
    bigquery.SchemaField('tiempo_dormido', 'FLOAT', mode='NULLABLE'),
    bigquery.SchemaField('tiempo_en_cama', 'FLOAT', mode='NULLABLE'),
    bigquery.SchemaField('eficiencia_sueno', 'FLOAT', mode='NULLABLE'),
    bigquery.SchemaField('porc_sueno_rem', 'FLOAT', mode='NULLABLE'),
    bigquery.SchemaField('porcentaje_sueno_profundo', 'FLOAT', mode='NULLABLE'),
    bigquery.SchemaField('ausencia_sueno_profundo', 'STRING', mode='NULLABLE'),
    bigquery.SchemaField('indice_microalertamientos', 'FLOAT', mode='NULLABLE'),
    bigquery.SchemaField('iah', 'FLOAT', mode='NULLABLE'),
    bigquery.SchemaField('ido', 'FLOAT', mode='NULLABLE'),
    bigquery.SchemaField('tiempo_bajo_90so2', 'FLOAT', mode='NULLABLE'),
    bigquery.SchemaField('ct90', 'FLOAT', mode='NULLABLE'),
    bigquery.SchemaField('iac', 'FLOAT', mode='NULLABLE'),
    bigquery.SchemaField('iao', 'FLOAT', mode='NULLABLE'),
    bigquery.SchemaField('iam', 'FLOAT', mode='NULLABLE'),
    bigquery.SchemaField('ih', 'FLOAT', mode='NULLABLE')
]


In [13]:
import os
service_account_file = "/home/clisueno/secrets/observatorio-ino-1-78cfc246d28f-key.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = service_account_file

In [14]:
client = bigquery.Client()

In [15]:
tablej = 'observatorio-ino-1.clinica_sueno.dwh_basal'

In [ ]:
# OJO!  Ejecutar solamente para crear la tabla cuando no existe
'''
tables = bigquery.Table(tablej, schema=schemaj)
tables.time_partitioning = bigquery.TimePartitioning(
    type_=bigquery.TimePartitioningType.MONTH,
    field="fecha_proced",  # Nombre de la columna de fecha para la partición
    )
tables = client.create_table(tables)
print("Tabla creada: {}".format(tablej))'''

Tabla creada: observatorio-ino-1.clinica_sueno.dwh_basal


In [16]:

import pyarrow
job_config = bigquery.LoadJobConfig(
    schema=schemaj,
    write_disposition='WRITE_TRUNCATE'
)
job = client.load_table_from_dataframe(
    df, tablej, job_config=job_config
    )
job.result()

/home/clisueno/venv/lib/python3.12/site-packages/google/cloud/bigquery/_pandas_helpers.py:489: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=observatorio-ino-1, location=us-central1, id=1b145b1a-1b36-437e-be8c-edd62781587d>